In [0]:
import numpy as np
import pandas as pd
import torch
from torch import nn
from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch import optim
import torch.nn.functional as F
from google.colab import drive
import matplotlib.pyplot as plt

from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data.sampler import SubsetRandomSampler

In [2]:
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
path_prefix = '/content/drive/Shared drives/CS 230 Project (EEG)/'
#data = np.load(path_prefix + 's44_psd_data_multitaper.npy')
#data = np.load(path_prefix + 's44_psd_data_welch_nonaveraged.npy')

data1 = np.load(path_prefix + 'combined_subjects_psd_1.npy')
data2 = np.load(path_prefix + 'combined_subjects_psd_2.npy')
data3 = np.load(path_prefix + 'combined_subjects_psd_3.npy')
data5 = np.load(path_prefix + 'combined_subjects_psd_5.npy')

#labels = np.load(path_prefix + 's44_labels.npy')
labels = np.load(path_prefix + 'combined_subjects_labels.npy')

In [9]:
print(np.shape(labels))

(26065, 1)


In [6]:
data_full = np.zeros((2000+6000+2000+5000, np.shape(data1)[1], np.shape(data2)[2]))
labels_full = np.zeros((2000+6000+2000+5000, 1))
data_full[0:1999] = data1[0:1999]
labels_full[0:1999] = labels[0:1999]

data_full[2000:7999] = data2[2000:7999]
data_full[8000:9999] = data3[8000:9999]
data_full[10000:14999] = data5[18000:22999]

print(data1[2000])

[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]


In [0]:
data = np.reshape(data, (np.shape(data)[0], -1))

dataset_indices = np.arange(data.shape[0])

# set seed to obtain same results across trials
np.random.seed(0)

# shuffle data indices 
np.random.shuffle(dataset_indices)
train_indices = dataset_indices[ : 6 * data.shape[0] // 10]
valid_indices = dataset_indices[ 6 * data.shape[0] // 10 : 8 * data.shape[0] // 10]
test_indices = dataset_indices[ 8 * data.shape[0] // 10 :]

x_train = data[train_indices].astype(np.float32)
y_train = labels[train_indices]
x_valid = data[valid_indices].astype(np.float32)
y_valid = labels[valid_indices]
x_test  = data[test_indices].astype(np.float32)
y_test  = labels[test_indices]

y_train = y_train.squeeze()
y_valid = y_valid.squeeze()
y_test = y_test.squeeze()

In [0]:
# Convert Dataset to Tensors
x_train, y_train, x_valid, y_valid, x_test, y_test = map(
    torch.tensor, (x_train, y_train, x_valid, y_valid, x_test, y_test)
)
# Training TensorDataset
train_ds = TensorDataset(x_train, y_train)

# Validation TensorDataset
valid_ds = TensorDataset(x_valid, y_valid)

# Testing TensorDataset
test_ds = TensorDataset(x_test, y_test)

In [7]:
# Start Device
dev = torch.device(
    "cuda") if torch.cuda.is_available() else torch.device("cpu")

print(torch.cuda.is_available())

True


In [0]:
def get_data(train_ds, valid_ds, test_ds, bs):
    return (
        DataLoader(train_ds, batch_size=bs, shuffle=True),
        DataLoader(valid_ds, batch_size=bs * 2),
        DataLoader(test_ds, batch_size=bs * 2)
    )

def preprocess(x, y):
    return x.to(dev), y.to(dev)

class WrappedDataLoader:
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self):
        return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches:
            yield (self.func(*b)) 

class Lambda(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func

    def forward(self, x):
        return self.func(x)

In [0]:
def fit(model, trainloader, validloader, opt, loss_function, epochs):
  for epoch in range(epochs):
    train_loss, valid_loss = [], []
    model.train()

    for data, target in trainloader:
      data = data.view(data.shape[0], -1)
      opt.zero_grad()
      output = model(data) #Forward prop
      #if len(train_loss) == 0:
      #  print(output)
      #print(output.shape)
      #print(target.shape)
      loss = loss_function(output, target.unsqueeze(1).type_as(output)) #Calculate loss
      loss.backward() #Backward prop
      opt.step() #Weight optimization

      train_loss.append(loss.item())
    
    model.eval()
    for data, target in validloader:
      data = data.view(data.shape[0], -1)
      output = model(data)
      loss = loss_function(output, target.unsqueeze(1).type_as(output))
      valid_loss.append(loss.item())

    acc1, prec1, rec1, f11 = test(model, trainloader)
    acc2, prec2, rec2, f12 = test(model, validloader)
    #print("Train Acc: " + str(acc1) + ", Valid Acc: " + str(acc2))
    
    #print ("Epoch:", epoch, "Training Loss: ", np.mean(train_loss), "Valid Loss: ", np.mean(valid_loss))

In [0]:
bs = 64
lr = 0.0001
epochs = 10

#loss_func = nn.CrossEntropyLoss()
#loss_func = F.cross_entropy
loss_func = nn.BCEWithLogitsLoss()

train_dl, valid_dl, test_dl = get_data(train_ds, valid_ds, test_ds, bs)

# Training Data Loader
train_dl = WrappedDataLoader(train_dl, preprocess)

# Validation Data Loader
valid_dl = WrappedDataLoader(valid_dl, preprocess)

# Test Data Loader
test_dl = WrappedDataLoader(test_dl, preprocess)

input_size = 150

# model = nn.Sequential(
#     nn.Linear(input_size, layer2),
#     nn.BatchNorm1d(layer2),
#     nn.Tanh(),
#     nn.Dropout(p=0.5),
#     nn.Linear(layer2, 1)
# )

model = nn.Sequential(
    nn.Linear(input_size, layer2),
    nn.BatchNorm1d(layer2),
    nn.ReLU(),
    nn.Linear(layer2, layer3),
    nn.Dropout(p=0.5),
    nn.BatchNorm1d(layer3),
    nn.ReLU(),
    nn.Dropout(p=0.5),
    nn.Linear(layer3, 1)

    #Lambda(lambda x: x.view(x.size(0), -1))
)

# model = nn.Sequential(
#     nn.Linear(input_size, layer2),
#     nn.BatchNorm1d(layer2),
#     nn.ReLU(),
#     nn.Linear(layer2, layer3),
#     nn.BatchNorm1d(layer3),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),
#     nn.Linear(layer3, layer4),
#     nn.BatchNorm1d(layer4),
#     nn.ReLU(),
#     nn.Dropout(p=0.5),
#     nn.Linear(layer4, 1)

#     #Lambda(lambda x: x.view(x.size(0), -1))
# )
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [72]:
10**(np.random.uniform(low=-6, high=-1))

3.4293590459565007e-06

In [0]:
bs = 64
train_dl, valid_dl, test_dl = get_data(train_ds, valid_ds, test_ds, bs)

# Training Data Loader
train_dl = WrappedDataLoader(train_dl, preprocess)

# Validation Data Loader
valid_dl = WrappedDataLoader(valid_dl, preprocess)

# Test Data Loader
test_dl = WrappedDataLoader(test_dl, preprocess)

In [78]:
#loss_func = nn.BCEWithLogitsLoss()
loss_func = nn.BCELoss()

iterations = 10
models = []
good_models = {}
for i in range(iterations):
  input_size  =150
  layer2 = int(np.random.uniform(low=input_size, high=2000))
  layer3 = int(np.random.uniform(low=10, high=layer2))
  layer4 = int(np.random.uniform(low=8, high = layer3))
  dropout = np.random.uniform(low=0, high=0.9)
  momentum = np.random.uniform(low=0.5, high=0.99)
  lr = 10**(np.random.uniform(low=-6, high=-4))
  epochs = int(np.random.uniform(low=5, high=30))

  #print("Layer2: " + str(layer2) + ", Layer3: " + str(layer3) + ", Drop: " + str(dropout) + ", Momentum: " + str(momentum) + ", LR: " + str(lr) + ", Epochs: " + str(epochs)) 
  model = nn.Sequential(
      nn.Linear(input_size, layer2),
      nn.BatchNorm1d(layer2),
      nn.ReLU(),
      nn.Linear(layer2, layer3),
      nn.BatchNorm1d(layer3),
      nn.ReLU(),
      nn.Dropout(p=dropout),
      nn.Linear(layer3, 1),
      nn.Sigmoid()
  )

  '''model = nn.Sequential(
      nn.Linear(input_size, layer2),
      nn.BatchNorm1d(layer2),
      nn.ReLU(),
      nn.Linear(layer2, layer3),
      nn.BatchNorm1d(layer3),
      nn.ReLU(),
      nn.Linear(layer3, layer4),
      nn.BatchNorm1d(layer4),
      nn.ReLU(),
      nn.Dropout(p=dropout),
      nn.Linear(layer4, 1),
      nn.Sigmoid()
  )'''

  '''model = nn.Sequential(
      nn.Linear(input_size, layer2),
      nn.BatchNorm1d(layer2),
      nn.ReLU(),
      nn.Dropout(p=dropout),
      nn.Linear(layer2, 1),
      nn.Sigmoid()
  )'''

  #opt = optim.SGD(model.parameters(), lr=lr, momentum=momentum)
  opt = optim.Adam(model.parameters(), lr=lr)
  model.to(dev)
  fit(model, train_dl, valid_dl, opt, loss_func, epochs)
  models.append(model)
  acc, prec, rec, f1 = test(model, test_dl)
  print(str(i) + " - Test:  Acc: " + str(acc) + ", Prec: "  + str(prec) + ", Rec: " + str(rec) + ", F1: " + str(f1))
  if acc > 0.75:
    #print("Good model")
    good_models[i] = {'layer2':layer2, 'layer3':layer3, 'layer4':layer4, 'dropout':dropout, 'momentum':momentum, 'lr':lr, 'epochs':epochs, 'acc':acc, 'prec':prec, 'rec':rec, 'f1':f1}

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in long_scalars


0 - Test:  Acc: 0.6948051948051948, Prec: 0.7417582417582418, Rec: 0.5895196506550219, F1: 0.656934306569343
1 - Test:  Acc: 0.6731601731601732, Prec: 0.7096774193548387, Rec: 0.5764192139737991, F1: 0.636144578313253
2 - Test:  Acc: 0.696969696969697, Prec: 0.7870967741935484, Rec: 0.5327510917030568, F1: 0.6354166666666666
3 - Test:  Acc: 0.7683982683982684, Prec: 0.7932692307692307, Rec: 0.7205240174672489, F1: 0.7551487414187643
4 - Test:  Acc: 0.6536796536796536, Prec: 0.6408163265306123, Rec: 0.6855895196506551, F1: 0.6624472573839663
5 - Test:  Acc: 0.6688311688311688, Prec: 0.7134831460674157, Rec: 0.5545851528384279, F1: 0.624078624078624
6 - Test:  Acc: 0.7272727272727273, Prec: 0.7562189054726368, Rec: 0.6637554585152838, F1: 0.7069767441860465
7 - Test:  Acc: 0.7186147186147186, Prec: 0.7647058823529411, Rec: 0.6244541484716157, F1: 0.6875
8 - Test:  Acc: 0.683982683982684, Prec: 0.7292817679558011, Rec: 0.5764192139737991, F1: 0.6439024390243903
9 - Test:  Acc: 0.744588744

In [72]:
for good in good_models:
  if good_models[good]['acc'] > 0.77:
    print(str(good) + ": Acc = " + str(good_models[good]['acc']) + ", F1: " + str(good_models[good]['f1']))
    print(good_models[good])
    print("")

223: Acc = 0.7705627705627706, F1: 0.7623318385650224
{'layer2': 1886, 'layer3': 1774, 'dropout': 0.00498994913304267, 'momentum': 0.9196005173910593, 'lr': 1.458088704342861e-05, 'epochs': 15, 'acc': 0.7705627705627706, 'prec': 0.783410138248848, 'rec': 0.74235807860262, 'f1': 0.7623318385650224}

348: Acc = 0.7770562770562771, F1: 0.7674943566591422
{'layer2': 1657, 'layer3': 1574, 'dropout': 0.6659056202731097, 'momentum': 0.5917465573996724, 'lr': 2.1763564709255216e-05, 'epochs': 21, 'acc': 0.7770562770562771, 'prec': 0.794392523364486, 'rec': 0.74235807860262, 'f1': 0.7674943566591422}

423: Acc = 0.7748917748917749, F1: 0.7570093457943925
{'layer2': 1468, 'layer3': 1254, 'dropout': 0.3001851913881326, 'momentum': 0.5666000743592952, 'lr': 1.49444389201979e-05, 'epochs': 17, 'acc': 0.7748917748917749, 'prec': 0.8140703517587939, 'rec': 0.7074235807860262, 'f1': 0.7570093457943925}

589: Acc = 0.7727272727272727, F1: 0.7608200455580867
{'layer2': 976, 'layer3': 714, 'dropout': 0.1

In [71]:
model = models[348]
print('Test Accuracy:')
acc, prec, rec, f1 = test(model, test_dl)
print("   Acc: " + str(acc) + ", Prec: "  + str(prec) + ", Rec: " + str(rec) + ", F1: " + str(f1))

Test Accuracy:
   Acc: 0.7770562770562771, Prec: 0.794392523364486, Rec: 0.74235807860262, F1: 0.7674943566591422


In [73]:
PATH = './ann_eeg_bce_rand7.pth'
print('Save state of network...')
answer = None
while answer not in ("Y", "N"):
    answer = input("Enter Y or N: ")
    if answer == "Y": # Save the current state of the network.
       
        torch.save(model.state_dict(), PATH)
        print('State of the network was saved.')
    elif answer == "N": # Move on without saving.
        print('State of the network was NOT saved.')
        continue
    else:
        print("Please enter yes or no.")

Save state of network...
Enter Y or N: Y
State of the network was saved.


In [0]:
def test(model, test_dl):
    nums = 0
    correct_predictions = 0

    #Precision - how many positive predictions were correct 
    pos_pred = 0
    pos_true = 0

    #Recall - how many real positive values were detected
    pos_all = 0
    
    for xb, yb in test_dl:
        raw_predict = model.forward(xb)
        #print(raw_predict)
        predictions = torch.round(raw_predict).squeeze()
        #print(predictions.size())
        predictions[predictions!=0] = 1
        #print("Predictions: " + str(predictions.squeeze()))

        #Calculate precision
        pred_pos = (predictions == 1)
        pred_pos = torch.sum(pred_pos)
        true_pos = (predictions == 1) * (yb == 1)
        #print(predictions.size())
        #print(yb.size())
        #print(true_pos.size())
        #print("X: " + str(true_pos))
        true_pos = torch.sum(true_pos)
        pos_pred += pred_pos.cpu().numpy()
        pos_true += true_pos.cpu().numpy()

        #print("True Positives: " + str(true_pos.cpu().numpy()))
        #print("Positive Predictions: " + str(pred_pos.cpu().numpy()))

        #Calculate recall
        all_pos = (yb == 1)
        all_pos = torch.sum(all_pos)
        pos_all += all_pos.cpu().numpy()

        #print("Actual Positives: " + str(all_pos.cpu().numpy()))

        compare = (predictions == yb)
        correct = torch.sum(compare)
        correct_predictions = correct_predictions + correct
        nums = nums + yb.shape[0]
        #print(correct_predictions)
    test_accuracy = correct_predictions.cpu().numpy() / nums
    precision = pos_true/pos_pred
    recall = pos_true/pos_all
    f1 = 2*precision*recall/(precision + recall)
    #print("Accuracy: " + str(test_accuracy))
    #print("Precision: " + str(precision))
    #print("Recall: " + str(recall))
    #print("F1: " + str(f1))

    return test_accuracy, precision, recall, f1